# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
file_path = "../output_data/cities.csv"
weather_df = pd.read_csv(file_path)
weather_df.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,mataura,NZ,-46.1927,168.8643,45.48,73,94,0.65,1631581970
1,ushuaia,AR,-54.8000,-68.3000,40.66,81,40,14.97,1631581889
2,rudnogorsk,RU,57.2639,103.7519,42.44,82,100,0.96,1631581971
3,hilo,US,19.7297,-155.0900,83.93,71,40,10.36,1631581856
4,makaryev,RU,57.8797,43.8069,56.30,93,100,11.88,1631581971


In [13]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        570 non-null    object 
 1   Country     568 non-null    object 
 2   Latitude    570 non-null    float64
 3   Longitude   570 non-null    float64
 4   Max Temp    570 non-null    float64
 5   Humidity    570 non-null    int64  
 6   Cloudiness  570 non-null    int64  
 7   Wind Speed  570 non-null    float64
 8   Date        570 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 40.2+ KB


In [14]:
weather_df.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,5.700000e+02
mean,20.285446,18.414949,66.180123,70.305263,52.756140,7.453789,1.631582e+09
std,33.032359,90.942759,14.313901,20.428145,41.264314,5.468790,7.616664e+01
min,-54.800000,-175.200000,21.360000,10.000000,0.000000,0.000000,1.631581e+09
25%,-7.942925,-62.196500,55.205000,60.000000,6.000000,3.440000,1.631582e+09
50%,26.142100,23.082200,67.550000,75.000000,55.000000,6.040000,1.631582e+09
75%,46.943325,103.479400,78.030000,85.000000,100.000000,10.540000,1.631582e+09
max,78.218600,179.316700,99.630000,100.000000,100.000000,30.760000,1.631582e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key = g_key)

figure_layout = {
    'width': '600px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating = True, max_intensity = max(humidity), point_radius = 6)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
nice_weather = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]
nice_weather.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
9,caravelas,BR,-17.7125,-39.2481,73.94,83,0,9.26,1631581972
49,khuzdar,PK,27.7384,66.6434,78.84,45,0,0.89,1631581982
74,gat,IL,31.6100,34.7642,79.39,70,0,6.73,1631581990
166,avera,US,33.1940,-82.5271,77.72,57,0,2.89,1631582016
240,port said,EG,31.2565,32.2841,78.67,72,0,9.24,1631582036


In [17]:
nice_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 9 to 503
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        14 non-null     object 
 1   Country     14 non-null     object 
 2   Latitude    14 non-null     float64
 3   Longitude   14 non-null     float64
 4   Max Temp    14 non-null     float64
 5   Humidity    14 non-null     int64  
 6   Cloudiness  14 non-null     int64  
 7   Wind Speed  14 non-null     float64
 8   Date        14 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 1.1+ KB


In [18]:
nice_weather.describe()

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,14.000000,14.000000,14.000000,14.000000,14.0,14.000000,1.400000e+01
mean,19.816443,33.453764,76.242143,56.214286,0.0,5.730000,1.631582e+09
std,22.304517,64.935812,2.933539,25.081954,0.0,3.112533,4.234922e+01
min,-25.298600,-82.527100,70.020000,12.000000,0.0,0.890000,1.631582e+09
25%,17.843500,-2.101975,74.367500,38.250000,0.0,3.015000,1.631582e+09
50%,30.613100,37.734200,77.315000,67.000000,0.0,5.635000,1.631582e+09
75%,33.240350,64.213650,78.662500,71.500000,0.0,8.665000,1.631582e+09
max,39.244600,152.853500,79.390000,94.000000,0.0,9.730000,1.631582e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = nice_weather.head(10).reset_index()

params = {
    "rankby": "distance",
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for {row['City']} - {index+1} out of {len(hotel_df)}.")
    response = requests.get(base_url, params = params).json()

    try:
        print(f"Closest hotel is {response['results'][0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        print(f"{row['City']} missing...")

Retrieving Results for caravelas - 1 out of 10.
Closest hotel is Cantinho da Beth.
Retrieving Results for khuzdar - 2 out of 10.
Closest hotel is jhalwan Rent A Car Moola Chotok Boking Office.
Retrieving Results for gat - 3 out of 10.
Closest hotel is מגדלי סמדר.
Retrieving Results for avera - 4 out of 10.
avera missing...
Retrieving Results for port said - 5 out of 10.
Closest hotel is أبراج اللؤلؤة.
Retrieving Results for gazanjyk - 6 out of 10.
Closest hotel is Railway motel.
Retrieving Results for gujar khan - 7 out of 10.
Closest hotel is Tanzeem e Islami Halqa Gujar Khan.
Retrieving Results for le vauclin - 8 out of 10.
Closest hotel is Villa Toscane.
Retrieving Results for hervey bay - 9 out of 10.
Closest hotel is Australiana Vlge Cvan Pk.
Retrieving Results for suixi - 10 out of 10.
Closest hotel is Xinhua Hotel.


In [21]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…